In [1]:
import os
print(os.getcwd())

e:\YangYueHan\project\MCLP\model


In [2]:
dataset_path = '../data/MP'

In [3]:
from tools import get_mapper

In [5]:
get_mapper(dataset_path=dataset_path)


*** Please check the corresponding dataset configuration yml file. ***
unique location num: 20607
unique user num: 10000


In [6]:
import numpy as np


- np.load()
- data_structure: item()

In [7]:
location_mapper = np.load(os.path.join(dataset_path, 'location_mapper.npy'), allow_pickle=True).item()

- dict: items()

In [8]:
for i, (loc, idx) in enumerate(list(location_mapper.items())[:5]):
    print(f'{i}: {loc} -> {idx}')

0: 9605 -> 0
1: 2085 -> 1
2: 6539 -> 2
3: 12587 -> 3
4: 12500 -> 4


In [1]:
from dataloader import datetime_to_features

In [2]:
weekday, hour = datetime_to_features(1672567200000)

In [3]:
print(weekday)
print(hour)

6
18


In [1]:
'for a and the'.split()

['for', 'a', 'and', 'the']

In [4]:
import torch
import gensim
from gensim import models

In [5]:
user_loc_matrix = torch.tensor([[2,1,3],[1,3,4],[4,5,6]])


In [6]:
num_users, num_locations = user_loc_matrix.shape
print(num_users, num_locations)

3 3


In [8]:
dictionary = gensim.corpora.Dictionary([[str(i)] for i in range(num_locations)])
dictionary.token2id

{'0': 0, '1': 1, '2': 2}

In [9]:
corpus = []
for user in user_loc_matrix:
    user_doc = [str(loc) for loc, count in enumerate(user) for _ in range(int(count))]
    corpus.append(dictionary.doc2bow(user_doc))
corpus

[[(0, 2), (1, 1), (2, 3)], [(0, 1), (1, 3), (2, 4)], [(0, 4), (1, 5), (2, 6)]]

In [10]:
lda = models.LdaModel(corpus, num_topics=3, random_state=42)

In [12]:
import numpy as np

In [13]:
user_topics = np.zeros((num_users, 3))
for i, user in enumerate(user_loc_matrix):
    user_doc = [str(loc) for loc, count in enumerate(user) for _ in range(int(count))]
    for item in lda[dictionary.doc2bow(user_doc)]:
        j = item[0]
        prob = item[1]
        # user_topic: user i在location j的概率
        user_topics[i, j] = prob
user_topics

array([[0.05799221, 0.88133264, 0.06067515],
       [0.04220628, 0.9138909 , 0.04390279],
       [0.0242673 , 0.95064938, 0.02508333]])

In [14]:
user_doc = [str(loc) for loc, count in enumerate(user_loc_matrix[0]) for _ in range(int(count))]
user_doc

['0', '0', '1', '2', '2', '2']

In [15]:
user_doc_embedding = dictionary.doc2bow(user_doc)
user_doc_embedding

[(0, 2), (1, 1), (2, 3)]

In [17]:
lda[user_doc_embedding]

[(0, 0.05799002), (1, 0.88139576), (2, 0.0606142)]